In [1]:
import matplotlib
matplotlib.use("Agg")

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [3]:
from sklearn.model_selection import train_test_split
from smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random

In [4]:
'''It is a built-in module in python that we've used to
make random numbers.'''

import cv2
'''OpenCV-Python is a library of Python designed
to solve computer vision problems.'''

import os
import glob
'''the glob module is used to retrieve files/pathnames
matching a specified pattern. It is also predicted that
according to benchmarks it is faster than other methods to
match pathnames in directories.'''

'the glob module is used to retrieve files/pathnames\nmatching a specified pattern. It is also predicted that\naccording to benchmarks it is faster than other methods to\nmatch pathnames in directories.'

In [5]:
# handle command line arguments
# '''it takes a list of strings and produce paths to input/output
# datasets or output accuracy/loss plot'''

# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
# 	help="path to input dataset (i.e., directory of images)")

# ap.add_argument("-m", "--model", type=str, default="gender_detection.model",
# 	help="path to output model")

# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output accuracy/loss plot")
# args = ap.parse_args()

In [6]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

# load image files recursively from the dataset

dataset_dir = "C:/Users/hp/Desktop/Projects/gender_classsss/gender_dataset_face"

# List all directories in the dataset directory
class_dirs = [class_dir for class_dir in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, class_dir))]

# Iterate over each directory and list all files within them
# image_files will contain the paths of all the images

image_files = []
for class_dir in class_dirs:
    for file in os.listdir(os.path.join(dataset_dir, class_dir)):
        if not file.startswith('.'):
            image_files.append(os.path.join(dataset_dir, class_dir, file))


# dataset_dir = "C:/Users/hp/Desktop/Projects/gender_classsss/gender_dataset_face"
# image_files = [os.path.join(dataset_dir, class_dir, file) 
#                for class_dir in os.listdir(dataset_dir)
#                for file in os.listdir(os.path.join(dataset_dir, class_dir))]

# image_files = [f for f in glob.glob(args.dataset + "/**/*", recursive=True) if not os.path.isdir(f)] 

random.seed(42)
'''Shuffle a list (reorganize the order of the list items)'''
random.shuffle(image_files)

In [7]:
len(image_files)

2307

In [8]:
# create ground-truth label from the image path

for img in image_files:

   #cv2.imread() method loads an image from the specified file  
    image = cv2.imread(img)

    #changing dimensions
    image = cv2.resize(image, (img_dims[0],img_dims[1]))

    '''img_to_array() function is provided by keras to convert
    a loaded image in PIL format into a NumPy
    array.'''
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2]
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label])

In [9]:
# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [10]:
# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [11]:
# augmenting dataset

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [12]:
# build model
model = SmallerVGGNet.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

from tensorflow.keras.optimizers import Adam

# compile the model
opt = Adam(learning_rate=lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [13]:
# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)



C:\Users\hp\AppData\Local\Temp\ipykernel_11940\57631111.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),


Epoch 1/100
28/28 [==============================] - 52s 1s/step - loss: 0.7661 - accuracy: 0.7294 - val_loss: 0.6577 - val_accuracy: 0.6580
Epoch 2/100
28/28 [==============================] - 37s 1s/step - loss: 0.5084 - accuracy: 0.8136 - val_loss: 1.1189 - val_accuracy: 0.4978
Epoch 3/100
28/28 [==============================] - 42s 1s/step - loss: 0.4503 - accuracy: 0.8310 - val_loss: 1.2630 - val_accuracy: 0.4978
Epoch 4/100
28/28 [==============================] - 43s 2s/step - loss: 0.4190 - accuracy: 0.8417 - val_loss: 1.3692 - val_accuracy: 0.4978
Epoch 5/100
28/28 [==============================] - 40s 1s/step - loss: 0.3786 - accuracy: 0.8630 - val_loss: 2.9209 - val_accuracy: 0.4978
Epoch 6/100
28/28 [==============================] - 40s 1s/step - loss: 0.3547 - accuracy: 0.8731 - val_loss: 3.1537 - val_accuracy: 0.4978
Epoch 7/100
28/28 [==============================] - 36s 1s/step - loss: 0.2869 - accuracy: 0.8989 - val_loss: 3.3775 - val_accuracy: 0.4978
Epoch 8/100
2

28/28 [==============================] - 24s 835ms/step - loss: 0.0516 - accuracy: 0.9815 - val_loss: 0.1989 - val_accuracy: 0.9156
Epoch 59/100
28/28 [==============================] - 28s 985ms/step - loss: 0.0595 - accuracy: 0.9753 - val_loss: 0.1144 - val_accuracy: 0.9567
Epoch 60/100
28/28 [==============================] - 34s 1s/step - loss: 0.0710 - accuracy: 0.9787 - val_loss: 0.1000 - val_accuracy: 0.9740
Epoch 61/100
28/28 [==============================] - 28s 1s/step - loss: 0.0772 - accuracy: 0.9719 - val_loss: 0.1085 - val_accuracy: 0.9524
Epoch 62/100
28/28 [==============================] - 25s 877ms/step - loss: 0.0593 - accuracy: 0.9809 - val_loss: 0.0773 - val_accuracy: 0.9762
Epoch 63/100
28/28 [==============================] - 25s 874ms/step - loss: 0.0462 - accuracy: 0.9826 - val_loss: 0.1101 - val_accuracy: 0.9740
Epoch 64/100
28/28 [==============================] - 26s 939ms/step - loss: 0.0579 - accuracy: 0.9775 - val_loss: 0.7027 - val_accuracy: 0.8355
Epoc

In [14]:
# save the model to disk
model.save('my_model.h5')
# model.save("C:/Users/hp/Desktop/Projects/gender_classsss/")

C:\Users\hp\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Save only the model architecture to a JSON file
model_json = model.to_json()
with open("my_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Save only the model weights to a separate file
model.save_weights('my_model_weights.h5')

In [ ]:
# Save the optimizer's state
model.save_weights('my_model_optimizer_weights.h5')

In [ ]:
plt.plot(H.history['accuracy'], label = 'accuracy')
plt.plot(H.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

In [26]:
# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0,N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0,N), H.history["val_accuracy"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")

In [28]:
# save plot to disk
plt.savefig("C:/Users/hp/Desktop/Projects/gender_classsss")

In [ ]:
# Load model architecture from JSON file
with open('my_model.json', 'r') as json_file:
    model_json = json_file.read()
loaded_model = tf.keras.models.model_from_json(model_json)

# Load weights into the new model
loaded_model.load_weights('my_model_weights.h5')
